## Production

Se pueden añadir parámetros
- Cubrecierre [0, 1]: Área y peso
- Cuello [0,1]: Área y peso
- Tipo de tela

In [1]:
import pandas as pd

data = pd.read_csv('./data.csv')
data.head()

,ancho_cm,largo_m,peso_g_m2,proveedor,cubre_cierre,talla_prenda,prendas_totales
0,140,48.6,130,A,0,1,91
1,140,103.2,90,A,0,1,137
2,140,80.4,160,A,0,1,110
3,120,137.8,200,A,0,1,168
4,140,101.6,200,A,0,1,141


In [2]:
print("Null values:", data.isnull().sum(), '\n')
print("Unique values:", data.nunique(), '\n')
print("Proveedors:", data.proveedor.unique(), '\n')
print("Values per proveedor:", data.proveedor.value_counts(), '\n')
print("Data types:", data.dtypes)

Null values: ancho_cm           0
largo_m            0
peso_g_m2          0
proveedor          0
cubre_cierre       0
talla_prenda       0
prendas_totales    0
dtype: int64 

Unique values: ancho_cm              4
largo_m            1300
peso_g_m2             8
proveedor             3
cubre_cierre          2
talla_prenda          3
prendas_totales     153
dtype: int64 

Proveedors: ['A' 'B' 'C'] 

Values per proveedor: proveedor
A    3000
B    3000
C    3000
Name: count, dtype: int64 

Data types: ancho_cm             int64
largo_m            float64
peso_g_m2            int64
proveedor           object
cubre_cierre         int64
talla_prenda         int64
prendas_totales      int64
dtype: object


## Artificial Neural Network

In [3]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def save_objects(filenames, object):
    if len(filenames) != len(object):
        raise Exception("Filenames and objects must be the same length")
        
    for index in range(len(filenames)):
        with open(filenames[index], 'wb') as f:
            pickle.dump(object[index], f)

In [4]:
data = pd.read_csv('./data.csv')

X = data[['prendas_totales', 'talla_prenda', 'cubre_cierre', 'proveedor']]
y = data[['peso_g_m2', 'largo_m', 'ancho_cm']]

ohe = OneHotEncoder(sparse_output = False)
providers = ohe.fit_transform(X[['proveedor', 'talla_prenda']])
X_categorical = pd.DataFrame(providers, columns = ohe.get_feature_names_out(['proveedor', 'talla_prenda']))

X = pd.concat([X.drop(columns = ['proveedor', 'talla_prenda']), X_categorical], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
y_train = scaler_y.fit_transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = scaler_y.transform(y_test)

input_dim = X.shape[1]
output_dim = y.shape[1]

save_objects(['scaler_x.pkl', 'scaler_y.pkl', 'ohe_inverse.pkl'], [scaler_x, scaler_y, ohe])

print(f"Input dimension: {input_dim}")
print(f"Output dimension: {output_dim}")

X.head()

Input dimension: 8
Output dimension: 3


,prendas_totales,cubre_cierre,proveedor_A,proveedor_B,proveedor_C,talla_prenda_1,talla_prenda_2,talla_prenda_3
0,91,0,1.0,0.0,0.0,1.0,0.0,0.0
1,137,0,1.0,0.0,0.0,1.0,0.0,0.0
2,110,0,1.0,0.0,0.0,1.0,0.0,0.0
3,168,0,1.0,0.0,0.0,1.0,0.0,0.0
4,141,0,1.0,0.0,0.0,1.0,0.0,0.0


## Implementation

In [5]:
from datetime import datetime

import tensorflow as tf
import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Input

c:\Users\Estadistica\Desktop\Producción\venv\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Estadistica\Desktop\Producción\venv\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Estadistica\Desktop\Producción\venv\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Plea

In [6]:
model_inv = Sequential([
    Input(shape = (input_dim,)),
    Dense(64, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(output_dim, activation = 'linear') 
])

model_inv.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,283 (36.26 KB)

 Trainable params: 9,283 (36.26 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model_inv.compile(
    optimizer = keras.optimizers.Adam(
        learning_rate = 0.001,
        beta_1 = 0.9,
        beta_2 = 0.999,
        epsilon = 1e-7
    ),
    loss = "mean_absolute_error",
    metrics = ["mae"]
)

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    restore_best_weights = True
)

history = model_inv.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs = 200,
    batch_size = 32,
    callbacks = [early_stopping]
)

model_inv.save('model_inverse.h5')
print(f"Model saved at {datetime.now().strftime('%H:%M:%S')}")

Epoch 1/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6100 - mae: 0.6100 - val_loss: 0.5943 - val_mae: 0.5943
Epoch 2/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5837 - mae: 0.5837 - val_loss: 0.5952 - val_mae: 0.5952
Epoch 3/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5816 - mae: 0.5816 - val_loss: 0.5912 - val_mae: 0.5912
Epoch 4/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5805 - mae: 0.5805 - val_loss: 0.5920 - val_mae: 0.5920
Epoch 5/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5799 - mae: 0.5799 - val_loss: 0.5903 - val_mae: 0.5903
Epoch 6/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5795 - mae: 0.5795 - val_loss: 0.5917 - val_mae: 0.5917
Epoch 7/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5784 - mae: 0.5784 - val_loss: 0.5901 - val_mae: 0.5901
Epoch 8/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5778 - mae: 0.5778 - val_loss: 0.5907 - val_mae: 0.5907
Epoch 9/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

Model saved at 12:44:30


## Evaluation

In [8]:
model = tf.keras.models.load_model('model_inverse.h5')
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test MAE: {test_mae}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5875 - mean_absolute_error: 0.5875 
Test loss: 0.5874608755111694, Test MAE: 0.5874608755111694


In [9]:
new_observation = {
    'prendas_totales': 91,
    'talla_prenda': 1,
    'cubre_cierre': 0,
    'proveedor': 'A'
}

def preprocess(new_observation):
    
    new_observation = pd.DataFrame(new_observation, index=[0])
    categorical = ohe.transform(new_observation[['proveedor', 'talla_prenda']])
    categorical = pd.DataFrame(categorical, columns = ohe.get_feature_names_out(['proveedor', 'talla_prenda']))
    new_observation = pd.concat([new_observation.drop(columns = ['proveedor', 'talla_prenda']), categorical], axis = 1)

    new_observation = scaler_x.transform(new_observation)
    
    return new_observation

new_observation = preprocess(new_observation)

prediction = scaler_y.inverse_transform(model.predict(new_observation))

def proccess_result(result):
    result = [value.astype(float).round(2) for value in result[0]]
    return result

peso_g_m2 = proccess_result(prediction)[0]
largo_m = proccess_result(prediction)[1]
ancho_cm = proccess_result(prediction)[2]

print(f"Ancho: {ancho_cm} cm, Largo: {largo_m} m, Peso: {peso_g_m2} g/m2")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Ancho: 140.29 cm, Largo: 47.51 m, Peso: 161.14 g/m2
